In [1]:
import psycopg2

In [2]:
host="157.230.240.246"
database="learn_psycopg2"
user="iykra_trainee"
password="passwordiykra"

In [13]:
def create_tables(conn,cur):
    """ create tables in the PostgreSQL database"""
    commands = (
        """
        CREATE TABLE vendors (
            vendor_id SERIAL PRIMARY KEY,
            vendor_name VARCHAR(255) NOT NULL
        )
        """,
        """ CREATE TABLE parts (
                part_id SERIAL PRIMARY KEY,
                part_name VARCHAR(255) NOT NULL
                )
        """,
        """
        CREATE TABLE part_drawings (
                part_id INTEGER PRIMARY KEY,
                file_extension VARCHAR(5) NOT NULL,
                drawing_data BYTEA NOT NULL,
                FOREIGN KEY (part_id)
                REFERENCES parts (part_id)
                ON UPDATE CASCADE ON DELETE CASCADE
        )
        """,
        """
        CREATE TABLE vendor_parts (
                vendor_id INTEGER NOT NULL,
                part_id INTEGER NOT NULL,
                PRIMARY KEY (vendor_id , part_id),
                FOREIGN KEY (vendor_id)
                    REFERENCES vendors (vendor_id)
                    ON UPDATE CASCADE ON DELETE CASCADE,
                FOREIGN KEY (part_id)
                    REFERENCES parts (part_id)
                    ON UPDATE CASCADE ON DELETE CASCADE
        )
        """)
    for command in commands:
        cur.execute(command)
    cur.close()
    conn.commit()

In [12]:
conn = psycopg2.connect(host = host, database = database, user = user, password = password)
cur = conn.cursor()

In [26]:
def insert_vendor(vendor_name,conn,cur):
    """ insert a new vendor into the vendors table """
    sql = """INSERT INTO vendors(vendor_name)
             VALUES(%s) RETURNING vendor_id;"""
    vendor_id = None
    try:
        # execute the INSERT statement
        cur.execute(sql, (vendor_name,))
        # get the generated id back
        vendor_id = cur.fetchone()[0]
        # commit the changes to the database
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
 
    return vendor_id

In [27]:
def insert_vendor_list(vendor_list,conn,cur):
    """ insert multiple vendors into the vendors table  """
    sql = "INSERT INTO vendors(vendor_name) VALUES(%s)"
    try:
        # execute the INSERT statement
        cur.executemany(sql,vendor_list)
        # commit the changes to the database
        conn.commit()
        # close communication with the database
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

In [29]:
# insert one vendor
#insert_vendor("3M Co.",conn,cur)
# insert multiple vendors
insert_vendor_list([
    ('AKM Semiconductor Inc.',),
    ('Asahi Glass Co Ltd.',),
    ('Daikin Industries Ltd.',),
    ('Dynacast International Inc.',),
    ('Foster Electric Co. Ltd.',),
    ('Murata Manufacturing Co. Ltd.',)
],conn,cur)

In [33]:
conn = psycopg2.connect(host=host,database=database, user=user, password=password)
cur = conn.cursor()

In [34]:
def update_vendor(vendor_id, vendor_name,conn,cur):
    """ update vendor name based on the vendor id """
    sql = """ UPDATE vendors
                SET vendor_name = %s
                WHERE vendor_id = %s"""
    updated_rows = 0
    try:
        cur.execute(sql, (vendor_name, vendor_id))
        # get the number of updated rows
        updated_rows = cur.rowcount
        # Commit the changes to the database
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    
    return updated_rows

In [36]:
def get_vendors(con,cur):
    """ query data from the vendors table """
    try:
        cur.execute("SELECT vendor_id, vendor_name FROM vendors ORDER BY vendor_name")
        print("The number of parts: ", cur.rowcount)
        row = cur.fetchone()
 
        while row is not None:
            print(row)
            row = cur.fetchone()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

In [39]:
cur.execute("select * from public.vendors")
cur.fetchall()

[(16, 'AKM Semiconductor Inc.'),
 (17, 'Asahi Glass Co Ltd.'),
 (18, 'Daikin Industries Ltd.'),
 (19, 'Dynacast International Inc.'),
 (20, 'Foster Electric Co. Ltd.'),
 (21, 'Murata Manufacturing Co. Ltd.'),
 (15, '3M Corp')]

In [40]:
def delete_part(part_id):
    """ delete part by part id """
    conn = None
    rows_deleted = 0
    try:
        # execute the UPDATE  statement
        cur.execute("DELETE FROM parts WHERE part_id = %s", (part_id,))
        # get the number of updated rows
        rows_deleted = cur.rowcount
        # Commit the changes to the database
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
 
    return rows_deleted